<a href="https://colab.research.google.com/github/antbartash/muffins/blob/main/muffins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>